In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
#from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import train_test_split
#from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score



In [2]:
data = pd.read_csv('../../raw_data/latlong_ohe.csv')

In [3]:
# NO USAR PARA MANTENER EL DF COMPLETO
data = data.sample(50000)

In [4]:
#Borrar NaN
data = data.dropna()

In [5]:
data.head()

,MONTH,DAY,DAY_OF_WEEK,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,CANCELLED,ORIGIN_LONGITUDE,ORIGIN_LATITUDE,DESTINATION_LONGITUDE,...,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN
87162,5,26,2,232.0,1660,542,0,-122.30931,47.44898,-97.03720,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
577214,5,13,3,143.0,896,1403,0,-122.37484,37.61900,-106.60919,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
29142,2,10,2,232.0,1464,1517,0,-97.03720,32.89595,-122.37484,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2429,2,10,2,164.0,867,1539,0,-115.15233,36.08036,-122.30931,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
289581,1,29,4,212.0,1481,1652,0,-111.97777,40.78839,-83.34884,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
X = data.drop(columns={'CANCELLED'}, axis=1)
y = data['CANCELLED']


In [7]:
X.shape, y.shape

((49998, 24), (49998,))

In [8]:
# Split data into train, test and validation sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3)

In [9]:
#Standard Scaler
std_scaler = StandardScaler()
X_train_sc = std_scaler.fit_transform(X_train)
X_test_sc=std_scaler.transform(X_test)

In [10]:
# NOT USE YET
# Use the same function above for the validation set
"""
X_test, X_val, y_test, y_val = train_test_split(
    X_test, y_test, test_size = 0.5, random_state = 42  # TEST = 15%
)
"""

'\nX_test, X_val, y_test, y_val = train_test_split(\n    X_test, y_test, test_size = 0.5, random_state = 42  # TEST = 15%\n)\n'

In [11]:
model = GradientBoostingClassifier()

param_grid = {
    'learning_rate': [0.1, 0.01],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5]
}


In [12]:
# Realiza el GridSearch y cross validate
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='precision', verbose=1, n_jobs=-1)
grid_search.fit(X_train_sc, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.01], 'max_depth': [3, 4, 5],
                         'n_estimators': [100, 200, 300]},
             scoring='precision', verbose=1)

In [13]:
# Obtiene los mejores params y la precisión promedio en la validación cruzada
best_params = grid_search.best_params_
best_precision = grid_search.best_score_

print("Mejores params:", best_params)
print("Precision promedio en el cross validate:", best_precision)

Mejores params: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}
Precision promedio en el cross validate: 0.9450757575757576


In [14]:
# Crea un nuevo modelo con los mejores params
best_model = GradientBoostingClassifier(learning_rate=0.01, max_depth=3, n_estimators=100)

# Realiza cross validate en el modelo entrenado con los mejores params
cv_results = cross_val_score(best_model, X_train_sc, y_train, cv=5, scoring='precision')

In [15]:
# Calcula la precisión promedio en el cross validarte
precision_mean = cv_results.mean()

precision_mean

0.9450757575757576

In [16]:
best_model.fit(X_train_sc, y_train)


GradientBoostingClassifier(learning_rate=0.01)

In [17]:
y_pred = best_model.predict(X_test_sc)


In [18]:
# Calcula la precisión en el conjunto de prueba
precision = precision_score(y_test, y_pred)
precision


0.8636363636363636